In [14]:
from functions import (prep, folding, inbu, LSTM_model)

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from keras.models import load_model
from jellyfish import damerau_levenshtein_distance, levenshtein_distance
import distance
from sklearn import metrics
import os
%load_ext autoreload
%autoreload 2
np.random.seed(42)

In [15]:
source = "running-example"
flatten_by = 'Packages' # input("Enter the value for flatten_by (Orders, Items or Packages): ")
single_log = 'True' # input("Enter the value for single_log (True/False): ")
if flatten_by == 'Packages':
    complete = 'True'
else:
    complete = 'False' # input("Enter the value for complete (True/False): ")
testing_other_remaining = False
add_customer = 1
normalize = False
# Error handling for invalid input values
if flatten_by not in ['Orders', 'Items', 'Packages']:
    raise ValueError("Wrong Input: flatten_by must be one of ['Orders', 'Items', 'Packages']")

if single_log.lower() not in ['true', 'false', '1', '0']:
    raise ValueError("Wrong Input: single_log must be a boolean value (True/False)")

if complete.lower() not in ['true', 'false', '1', '0']:
    raise ValueError("Wrong Input: complete must be a boolean value (True/False)")

# Convert input values to boolean
single_log = single_log.lower() in ['true', '1']
complete = complete.lower() in ['true', '1'] 
if complete:
    csvname = flatten_by  + '_complete'
    fl = None
else:
    csvname = flatten_by  + '_filter'
    fl = prep.act_filter(flatten_by )

time_feat = ['Time_Diff', 'Time_Since_Start', 'Time_Since_Midnight','Weekday','Position']
other_features = [] + int(flatten_by != 'Items') * ['Amount_Items'] + int(flatten_by != 'Packages') * ['In_Package'] + int(flatten_by != 'Orders') * ['Amount_Orders']
drops_col_order = ["weight", "price", "Event_ID", 'Products']
print("Settings:")
print(f"flatten_by: {flatten_by}")
print(f"single_log: {single_log}")
print(f"complete: {complete}")

Settings:
flatten_by: Packages
single_log: True
complete: True


In [16]:
## prep the ocel and reading
ocel, act_dict, cust_dict = prep.prepare_flat_ocel(source, flatten_on= flatten_by , filter= fl)
print(act_dict)
print(cust_dict)
## create the enriched and some more preprocessing as well as saving the single and enriched versions
ocel = prep.gen_enriched_single_plus_csv(OCEL = ocel,flatted_by = flatten_by ,csvname = csvname, drops_col= drops_col_order, single=single_log)
## adding features
ocel =prep.generate_features(ocel, single= single_log)
## define some static variables 
divisor = np.mean(ocel['Time_Diff'])  # average time between events
divisor2 = np.mean(ocel['Time_Since_Start'])  # average time between current and first events
divisorTR = np.mean(ocel['Remaining_Time'])  # average time instance remaining
divisor3 = ocel.groupby('Case_ID')['Time_Since_Start'].apply(lambda x: (x.iloc[-1] - x).mean()).mean()

print(f"divisor: {divisor}")
print(f"divisor2: {divisor2}")
print(f"divisorTR: {divisorTR}")
print(f"divisor3: {divisor3}")
print(f'Amount of rows of the OCEL: {len(ocel)}')
#folding the data 
ocel_train, ocel_test = folding.folding_train_test(ocel,old_ver=False)

{'place order': 'A', 'pick item': 'B', 'confirm order': 'C', 'item out of stock': 'D', 'reorder item': 'E', 'pay order': 'F', 'create package': 'G', 'send package': 'H', 'failed delivery': 'I', 'package delivered': 'J', 'payment reminder': 'K'}
{'Marco Pegoraro': 'a', 'Gyunam Park': 'b', 'Majid Rafiei': 'c', 'Junxiong Gao': 'd', 'Seran Uysal': 'e', 'Christina Rensinghof': 'f', 'Wil van der Aalst': 'g', 'Christine Dobbert': 'h', 'Luis Santos': 'i', 'Kefang Ding': 'j', 'Mohammadreza Fani Sani': 'k', 'Tobias Brockhoff': 'l', 'Anahita Farhang Ghahfarokhi': 'm', 'Mahnaz Qafari': 'n', 'Claudia Graf': 'o', 'Mahsa Bafrani': 'p', 'Lisa Mannel': 'q'}
divisor: 47402.199267063676
divisor2: 83320.73980760422
divisorTR: 82686.42281264316
divisor3: 77428.97973944295
Amount of rows of the OCEL: 4366


In [17]:
act_feat = list(filter(lambda k: k.startswith('Act_') and not k.startswith('Next_Act_'), ocel_train.columns))
act_feat.remove('Act_!')
act_feat_dict = {index: value.replace('Act_', '') for index, value in enumerate(act_feat)}
target_act_feat = list(filter(lambda k: k.startswith('Next_Act_') and not k.startswith('Act_'), ocel_train.columns))
target_act_feat_dict = {index: value.replace('Next_Act_', '') for index, value in enumerate(target_act_feat)}

cust_feat = list(filter(lambda k: 'Cust_' in k, ocel_train.columns)) * (1 - int(single_log)) * add_customer

feature_select = act_feat + time_feat + other_features *(1 - int(single_log)) + cust_feat *(1 - int(single_log))
print(f"Length of act_feat: {len(act_feat)}, Length of cust_feat: {len(cust_feat)}")

## define dimensions of inputs
max_trace_length = prep.gen_traces_and_maxlength_of_trace(ocel)[1]
target_act_length = len(target_act_feat)
number_of_train_cases = len(ocel_train)
num_of_features = len(feature_select) if 'In_Package' not in ocel.columns else len(feature_select) - (len(ocel['In_Package'].unique()) == 1)
print(f"Number of train cases: {number_of_train_cases}, Max trace length: {max_trace_length}, Number of features: {num_of_features}")
print(target_act_feat_dict)
print(act_feat_dict)
print(cust_feat)
print(num_of_features)

Length of act_feat: 4, Length of cust_feat: 0
Number of train cases: 2913, Max trace length: 9, Number of features: 9
{0: '!', 1: 'G', 2: 'H', 3: 'I', 4: 'J'}
{0: 'G', 1: 'H', 2: 'I', 3: 'J'}
[]
9


In [18]:
X_train,y_train_a, y_train_t, y_train_tr = inbu.generating_inputs(OCEL=ocel_train,
                                                                  num_of_features=num_of_features,
                                                                  max_trace_length=max_trace_length,
                                                                  taf=target_act_feat,
                                                                  act=act_feat,
                                                                  custf=cust_feat,
                                                                  divisor_next=divisor,
                                                                  divisor_since=divisor2,
                                                                  divisor_remaining=divisorTR,
                                                                  normalize = normalize, 
                                                                  test = testing_other_remaining)

print(f"Shape of X_train: {X_train.shape}")
print(f"This matches the desired shape (number_of_train_cases, max_trace_length, num_of_features): {(number_of_train_cases, max_trace_length, num_of_features)} => {X_train.shape ==(number_of_train_cases, max_trace_length, num_of_features)}")
print(f"Shape of y_train_a: {y_train_a.shape}, this matches the desired shape (number_of_train_cases, target_act_length): {(number_of_train_cases, target_act_length)} => {y_train_a.shape ==(number_of_train_cases, target_act_length)}")
print(f"Shape of y_train_t: {y_train_t.shape}, this matches the desired shape (number_of_train_cases, ): {(number_of_train_cases, )} => {y_train_t.shape ==(number_of_train_cases, )}")
print(f"Shape of y_train_tr: {y_train_tr.shape}, this matches the desired shape (number_of_train_cases, ): {(number_of_train_cases, )} => {y_train_tr.shape ==(number_of_train_cases, )}")


Shape of X_train: (2913, 9, 9)
This matches the desired shape (number_of_train_cases, max_trace_length, num_of_features): (2913, 9, 9) => True
Shape of y_train_a: (2913, 5), this matches the desired shape (number_of_train_cases, target_act_length): (2913, 5) => True
Shape of y_train_t: (2913,), this matches the desired shape (number_of_train_cases, ): (2913,) => True
Shape of y_train_tr: (2913,), this matches the desired shape (number_of_train_cases, ): (2913,) => True


In [19]:
X_train[1][-1]

array([0.        , 1.        , 0.        , 0.        , 2.        ,
       0.2564016 , 0.14587004, 0.599294  , 3.        ], dtype=float32)

In [20]:
if single_log:
    model_file = csvname + '_single'
else:
    model_file = csvname + '_enriched'
print(model_file)

Packages_complete_single


In [ ]:
history, best_model_name, early_stopping= LSTM_model.LSTM_MODEL(X_train, y_train_a, y_train_t, y_train_tr,filename=model_file)

In [26]:
print(best_model_name.best)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
epoch = early_stopping.stopped_epoch - 50
val_loss = min(history.history['val_loss'])

modelname = model_file + '_{epoch:02d}-{val_loss:.2f}.h5'

NameError: name 'best_model_name' is not defined

In [27]:
modelname = 'model_Packages_complete_single_87-1.15.h5'

In [8]:
len(ocel_test)

1453

In [11]:
model = load_model(f'./output_files/models/{modelname}')
X_test,y_test_a, y_test_t, y_test_tr = inbu.generating_inputs(OCEL=ocel_test,
                                                                  num_of_features=num_of_features,
                                                                  max_trace_length=max_trace_length,
                                                                  taf=target_act_feat,
                                                                  act=act_feat,
                                                                  custf=cust_feat,
                                                                  divisor_next=divisor,
                                                                  divisor_since=divisor2,
                                                                  divisor_remaining=divisorTR, 
                                                                  normalize = normalize, 
                                                                  test = testing_other_remaining)

# y_t = y_t * divisor3

y = model.predict(X_test,verbose=1)
y_char = y[0][:][:]
y_t = y[1][:][:]
y_tr = y[2][:][:]
max_index_list = [np.argmax(pred) for pred in y_char]
pred_act_list = [target_act_feat_dict.get(item, item) for item in max_index_list]
y_t = np.maximum(y_t, 0)
y_t1 = y_t * divisor
y_tr1 = y_tr * divisorTR

columns_to_drop = [col for col in ocel_test.columns if 'Act_' in col] + \
                  [col for col in ocel_test.columns if 'Cust_' in col] + \
                  ['Customers', 'Next_Time_Since_Start',
                   'Next_Time_Since_Midnight', 'Next_Weekday',
                   'Position', 'Time_Since_Midnight', 'Weekday'] + other_features

columns_to_drop_existing = [col for col in columns_to_drop if col in ocel_test.columns]
output_ocel = ocel_test.drop(columns=columns_to_drop_existing).copy()
output_ocel['Pred_Activity'] = pred_act_list
output_ocel['Pred_Time_Diff'] = y_t1 
output_ocel['Pred_Remaining_Time'] = y_tr1 

output_ocel['Levenshtein'] = output_ocel.apply(lambda row: 1 - levenshtein_distance(row['Pred_Activity'], row['Next_Activity']), axis=1)
output_ocel['Damerau'] = output_ocel.apply(lambda row: 1 - (damerau_levenshtein_distance(row['Pred_Activity'], row['Next_Activity']) / max(len(row['Pred_Activity']), len(row['Next_Activity']))), axis=1)
output_ocel['Damerau'] = output_ocel['Damerau'].clip(lower=0)
output_ocel['Jaccard'] = output_ocel.apply(lambda row: 1 - distance.jaccard(row['Pred_Activity'], row['Next_Activity']), axis=1)

output_ocel
from sklearn import metrics
act_comp = output_ocel['Pred_Activity'] == output_ocel['Next_Activity'] 
print(f'The accuracy of the activation prediction is {sum(act_comp)/len(act_comp)}')
MAE_Time_diff = metrics.mean_absolute_error(output_ocel['Pred_Time_Diff']/ (24 * 60 * 60),output_ocel['Next_Time_Diff']/ (24 * 60 * 60))
MAE_rem_time = metrics.mean_absolute_error(output_ocel['Pred_Remaining_Time']/ (24 * 60 * 60),output_ocel['Next_Remaining_Time']/ (24 * 60 * 60))
RMSE_Time_diff = metrics.mean_squared_error(output_ocel['Pred_Time_Diff']/ (24 * 60 * 60),output_ocel['Next_Time_Diff']/ (24 * 60 * 60),squared=False)
RMSE_rem_time = metrics.mean_squared_error(output_ocel['Pred_Remaining_Time']/ (24 * 60 * 60),output_ocel['Next_Remaining_Time']/ (24 * 60 * 60),squared=False)
print(f'MAE of the time between events in days {MAE_Time_diff}')
print(f'MAE of the remaining time in days {MAE_rem_time}')
# print(f'RMSE of the time between events in days {RMSE_Time_diff}')
# print(f'RMSE of the remaining time in days {RMSE_rem_time}')

46/46 [==============================] - 1s 6ms/step
The accuracy of the activation prediction is 0.9091534755677908
MAE of the time between events in days 0.40873667434465477
MAE of the remaining time in days 0.362053561318983


In [16]:
output_ocel

,Case_ID,Activity,Timestamp,Time_Diff,Time_Since_Start,Remaining_Time,Next_Time_Diff,Next_Remaining_Time,Next_Activity,Trace_Len,Pred_Activity,Pred_Time_Diff,Pred_Remaining_Time,Levenshtein,Damerau,Jaccard
0,660002,G,2019-05-23 17:26:26,0,0,80898,1006.0,79892.0,H,3,H,57451.085938,58707.195312,1,1.0,1.0
1,660002,H,2019-05-23 17:43:12,1006,1006,79892,79892.0,0.0,J,3,J,59817.968750,34.501392,1,1.0,1.0
2,660002,J,2019-05-24 15:54:44,79892,80898,0,0.0,0.0,!,3,!,1.493572,11.478123,1,1.0,1.0
3,660005,G,2019-05-25 01:00:00,0,0,377436,242972.0,134464.0,H,3,H,104568.000000,49668.558594,1,1.0,1.0
4,660005,H,2019-05-27 20:29:32,242972,242972,134464,134464.0,0.0,J,3,J,63515.066406,30.554998,1,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,661323,H,2020-08-13 14:25:30,134730,134730,417041,417041.0,0.0,J,3,J,33010.410156,32.862690,1,1.0,1.0
1449,661323,J,2020-08-18 10:16:11,417041,551771,0,0.0,0.0,!,3,!,21.667925,17.966599,1,1.0,1.0
1450,661324,G,2020-08-17 01:00:00,0,0,233852,123069.0,110783.0,H,3,H,58256.566406,55050.621094,1,1.0,1.0
1451,661324,H,2020-08-18 11:11:09,123069,123069,110783,110783.0,0.0,J,3,J,16989.353516,27.330965,1,1.0,1.0


In [146]:
# Generate inputs with varying prefix lengths
prefix_lengths = range(2, max_trace_length-1)  # List of prefix lengths to consider
results = []

for prefix_length in prefix_lengths:
    print(f"Results for Prefix Length {prefix_length}:")
    
    # Generate inputs with the current prefix length
    X_test, y_test_a, y_test_t, y_test_tr = inbu.generating_inputs(OCEL=ocel_test,
                                                                  num_of_features=num_of_features,
                                                                  max_trace_length=max_trace_length,
                                                                  taf=target_act_feat,
                                                                  act=act_feat,
                                                                  custf=cust_feat,
                                                                  divisor_next=divisor,
                                                                  divisor_since=divisor2,
                                                                  divisor_remaining=divisorTR,
                                                                  normalize = normalize, 
                                                                  prefix_length=prefix_length,
                                                                  test = testing_other_remaining)

    # Make predictions with the model
    y = model.predict(X_test, verbose=1)
    y_char = y[0][:][:]
    y_t = y[1][:][:]
    y_tr = y[2][:][:]

    max_index_list = [np.argmax(pred) for pred in y_char]
    pred_act_list = [target_act_feat_dict.get(item, item) for item in max_index_list]
    y_t = np.maximum(y_t, 0)
    y_t1 = y_t * divisor
    y_tr1 = y_tr * divisorTR

    columns_to_drop = [col for col in ocel_test.columns if 'Act_' in col] + \
                    [col for col in ocel_test.columns if 'Cust_' in col] + \
                    ['Customers', 'Next_Time_Since_Start',
                    'Next_Time_Since_Midnight', 'Next_Weekday',
                    'Position', 'Time_Since_Midnight', 'Weekday'] + other_features

    columns_to_drop_existing = [col for col in columns_to_drop if col in ocel_test.columns]
    trace_length = ocel_test['Trace_Len'].values
    output_ocel = ocel_test[trace_length >= prefix_length].reset_index(drop= True).drop(columns=columns_to_drop_existing).copy()
    output_ocel['Pred_Activity'] = pred_act_list
    output_ocel['Pred_Time_Diff'] = y_t1
    output_ocel['Pred_Remaining_Time'] = y_tr1

    output_ocel['Levenshtein'] = output_ocel.apply(lambda row: 1 - levenshtein_distance(row['Pred_Activity'], row['Next_Activity']), axis=1)
    output_ocel['Damerau'] = output_ocel.apply(lambda row: 1 - (damerau_levenshtein_distance(row['Pred_Activity'], row['Next_Activity']) / max(len(row['Pred_Activity']), len(row['Next_Activity']))), axis=1)
    output_ocel['Damerau'] = output_ocel['Damerau'].clip(lower=0)
    output_ocel['Jaccard'] = output_ocel.apply(lambda row: 1 - distance.jaccard(row['Pred_Activity'], row['Next_Activity']), axis=1)

    mae_time_diff = metrics.mean_absolute_error(output_ocel['Pred_Time_Diff']/ (24 * 60 * 60), output_ocel['Next_Time_Diff']/ (24 * 60 * 60)) 
    mae_remaining_time = metrics.mean_absolute_error(output_ocel['Pred_Remaining_Time']/ (24 * 60 * 60), output_ocel['Next_Remaining_Time']/ (24 * 60 * 60)) 
    rmse_time_diff = metrics.mean_squared_error(output_ocel['Pred_Time_Diff']/ (24 * 60 * 60), output_ocel['Next_Time_Diff']/ (24 * 60 * 60), squared=False) 
    rmse_remaining_time = metrics.mean_squared_error(output_ocel['Pred_Remaining_Time']/ (24 * 60 * 60), output_ocel['Next_Remaining_Time']/ (24 * 60 * 60), squared=False) 
    act_comp = output_ocel['Pred_Activity'] == output_ocel['Next_Activity'] 
    # Store the results for the current prefix length
    results.append({
        'Prefix Length': prefix_length,
        'length': len(y_tr),
        'Accuracy_Activity_pred': sum(act_comp)/len(act_comp),
        'MAE_Time_Difference': mae_time_diff,
        'MAE_Remaining_Time': mae_remaining_time,
        'RMSE_Time_Difference': rmse_time_diff,
        'RMSE_Remaining_Time': rmse_remaining_time
    })

    # Output additional values based on Case_ID and prefix length
    for case_id in output_ocel['Case_ID'].unique():
        case_data = output_ocel[output_ocel['Case_ID'] == case_id]
        trace_length = len(case_data)
        if prefix_length <= trace_length:
            # print(f"\nAdditional values for Prefix Length {prefix_length} and Case ID {case_id}:")
            case_data_prefix = case_data[:prefix_length]
            #print(case_data_prefix.to_string(index=False))
            
            mae_time_diff_case = metrics.mean_absolute_error(case_data_prefix['Pred_Time_Diff']/ (24 * 60 * 60), case_data_prefix['Next_Time_Diff']/ (24 * 60 * 60))
            mae_remaining_time_case = metrics.mean_absolute_error(case_data_prefix['Pred_Remaining_Time']/ (24 * 60 * 60), case_data_prefix['Next_Remaining_Time']/ (24 * 60 * 60))
            rmse_time_diff_case = metrics.mean_squared_error(case_data_prefix['Pred_Time_Diff']/ (24 * 60 * 60), case_data_prefix['Next_Time_Diff']/ (24 * 60 * 60), squared=False)
            rmse_remaining_time_case = metrics.mean_squared_error(case_data_prefix['Pred_Remaining_Time']/ (24 * 60 * 60), case_data_prefix['Next_Remaining_Time']/ (24 * 60 * 60), squared=False)
            
            # print(f"\nMetrics for Prefix Length {prefix_length} and Case ID {case_id}:")
            # print("MAE Time Difference:", mae_time_diff_case)
            # print("MAE Remaining Time:", mae_remaining_time_case)
            # print("RMSE Time Difference:", rmse_time_diff_case)
            # print("RMSE Remaining Time:", rmse_remaining_time_case)

# Output the overall results
df_results = pd.DataFrame(results)
print("\nOverall Results:")
display(df_results)


Results for Prefix Length 2:
46/46 [==============================] - 0s 5ms/step
Results for Prefix Length 3:
46/46 [==============================] - 0s 8ms/step
Results for Prefix Length 4:
13/13 [==============================] - 0s 7ms/step
Results for Prefix Length 5:
6/6 [==============================] - 0s 7ms/step
Results for Prefix Length 6:
3/3 [==============================] - 0s 9ms/step
Results for Prefix Length 7:
1/1 [==============================] - 0s 26ms/step

Overall Results:


,Prefix Length,length,Accuracy_Activity_pred,MAE_Time_Difference,MAE_Remaining_Time,RMSE_Time_Difference,RMSE_Remaining_Time
0,2,1453,0.909153,0.400650,0.367541,0.827951,0.913050
1,3,1453,0.909153,0.406871,0.363698,0.851532,0.896965
2,4,385,0.657143,0.425315,0.744468,0.858440,1.432150
3,5,165,0.539394,0.477796,1.204472,0.927577,1.916912
4,6,75,0.546667,0.465628,1.407284,0.940390,2.194399
5,7,15,0.533333,0.421512,1.938391,0.797979,2.646663


In [149]:
df_results ['MAE_Remaining_Time'].mean()

1.0043091030377476

In [28]:
import pickle

# Sample data dictionary
data = {
    'num_of_features': num_of_features,
    'max_trace_length': max_trace_length,
    'target_act_feat': target_act_feat,
    'act_feat': act_feat,
    'cust_feat': cust_feat,
    'divisor': divisor,
    'divisor2': divisor2,
    'divisorTR': divisorTR,
    'single_log': single_log,
    'target_act_feat_dict': target_act_feat_dict,
    'modelname': modelname
}

# Write data to a file using pickle
with open('output_files/settings.pkl', 'wb') as file:
    pickle.dump(data, file)

# Read data from the file
with open('output_files/settings.pkl', 'rb') as file:
    loaded_data = pickle.load(file)

# Print the loaded data and their types
for key, value in loaded_data.items():
    print(f'{key}: {type(value)}')


num_of_features: <class 'int'>
max_trace_length: <class 'int'>
target_act_feat: <class 'list'>
act_feat: <class 'list'>
cust_feat: <class 'list'>
divisor: <class 'numpy.float64'>
divisor2: <class 'numpy.float64'>
divisorTR: <class 'numpy.float64'>
single_log: <class 'bool'>
target_act_feat_dict: <class 'dict'>
modelname: <class 'str'>
